In [ ]:
import pandas as pd

: 

In [ ]:
df = pd.read_csv("../base-dados/eventos-climaticos.csv")

In [ ]:
# Selecionar colunas essenciais
df_simplificado = df[[
    'Disaster Type', 'Start Year', 'Total Deaths', 'No. Injured',
    'Total Affected', 'Total Damage (\'000 US$)', 'Magnitude', 'Country'
]].copy()

# Criar coluna 'Nivel de Risco' com base em regras simples
def classificar_risco(row):
    impacto = 0
    if pd.notnull(row['Total Deaths']):
        impacto += row['Total Deaths'] * 5
    if pd.notnull(row['No. Injured']):
        impacto += row['No. Injured'] * 2
    if pd.notnull(row['Total Affected']):
        impacto += row['Total Affected'] * 1
    if pd.notnull(row['Total Damage (\'000 US$)']):
        impacto += row['Total Damage (\'000 US$)'] * 0.1

    if impacto == 0:
        return 'Muito Baixo'
    elif impacto <= 100:
        return 'Baixo'
    elif impacto <= 1000:
        return 'Médio'
    elif impacto <= 10000:
        return 'Alto'
    else:
        return 'Muito Alto'

df_simplificado['Nivel de Risco'] = df_simplificado.apply(classificar_risco, axis=1)

# Verificar as primeiras linhas da nova tabela
df_simplificado.head()


In [ ]:
df_simplificado.to_csv("../base-dados/eventos-climaticos-limpos.csv")

In [ ]:
df_model = pd.read_csv("../base-dados/eventos-climaticos-limpos.csv")

numerical_cols = [
    'Start Year', 'Total Deaths', 'No. Injured',
    'Total Affected', 'Total Damage (\'000 US$)', 'Magnitude'
]

df_model[numerical_cols] = df_model[numerical_cols].fillna(0)

df_model.head()

In [ ]:
df_model = df_model.drop("Unnamed: 0", axis=1)

In [ ]:
df_model

In [ ]:
df_model.info()

In [ ]:
y = df_model["Nivel de Risco"]
X = df_model.drop("Nivel de Risco", axis=1)
X = pd.get_dummies(X, columns=['Disaster Type', 'Country'])

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

In [ ]:
pd.to_pickle(clf, "../app/modelo.pkl")